In [30]:
import numpy as np
import pandas as pd
import polars as pl

import cate.dataset as cds
from cate.utils import path_linker

In [6]:
path_link = path_linker("test")

In [35]:
stats = {}

In [10]:
dataset_names = ("criteo", "hillstorm", "lenta", "megafon")
ds_list = [cds.Dataset.load(path_linker(name).mart) for name in dataset_names]

In [36]:
stats["count"] = [len(ds) for ds in ds_list]
stats["explanatory_count"] = [ds.X.shape[1] for ds in ds_list]
stats["conversion_rate"] = [np.mean(ds.y) for ds in ds_list]
stats["tg_rate"] = [np.mean(ds.w) for ds in ds_list]

In [37]:
ates = []
for ds in ds_list:
    ate = np.mean(ds.y[ds.w == 1]) - np.mean(ds.y[ds.w == 0])
    ates.append(ate)
stats["ate"] = ates

In [42]:
df = pd.DataFrame(stats, index=dataset_names)

In [43]:
df

,count,explanatory_count,conversion_rate,tg_rate,ate
criteo,13979592,12,0.002917,0.850000,0.001152
hillstorm,64000,12,0.009031,0.667094,0.004955
lenta,687029,195,0.108246,0.750903,0.007547
megafon,600000,50,0.204190,0.500613,0.049519
